In [1]:
from enchant.utils import levenshtein
import pandas as pd
import numpy as np
import copy

In [2]:
cps_df = pd.read_csv('clean_data/cellphones.csv')
tgdd_df = pd.read_csv('clean_data/tgdd.csv')
tiki_df = pd.read_csv('clean_data/tiki.csv')
vt_df = pd.read_csv('clean_data/viettel.csv')
ddv_df = pd.read_csv('clean_data/didongviet.csv')
dmx_df = pd.read_csv('clean_data/dienmayxanh.csv')

In [3]:
cps_column = []
tgdd_column = []
tiki_column = []
vt_column = []
ddv_column = []
dmx_column = []

for e in cps_df.columns:
    cps_column.append(e)

for e in tgdd_df.columns:
    tgdd_column.append(e)

for e in tiki_df.columns:
    tiki_column.append(e)

for e in vt_df.columns:
    vt_column.append(e)

for e in ddv_df.columns:
    ddv_column.append(e)

for e in dmx_df.columns:
    dmx_column.append(e)

In [4]:
def name_matcher(column1, column2):
    res = []
    for c1 in column1:
        tmp = []
        for c2 in column2:
            tmp.append(levenshtein(c1, c2))
        res.append(tmp)
    return res

def jaccard(A, B):
    nominator = A.symmetric_difference(B)
    denominator = A.union(B)
    distance = len(nominator)/len(denominator)    
    return distance

def instance_matcher(df1, df2, column1, column2):
    res = []
    for c1 in column1:
        tmp = []
        for c2 in column2:
            tmp.append(jaccard(set(df1[c1]), set(df2[c2])))
        res.append(tmp)
    return res

In [5]:
def check(engaged, guyprefers, galprefers):
    inverseengaged = dict((v,k) for k,v in engaged.items())
    for she, he in engaged.items():
        shelikes = galprefers[she]
        shelikesbetter = shelikes[:shelikes.index(he)]
        helikes = guyprefers[he]
        helikesbetter = helikes[:helikes.index(she)]
        for guy in shelikesbetter:
            guysgirl = inverseengaged[guy]
            guylikes = guyprefers[guy]
            if guylikes.index(guysgirl) > guylikes.index(she):
                return False
        for gal in helikesbetter:
            girlsguy = engaged[gal]
            gallikes = galprefers[gal]
            if gallikes.index(girlsguy) > gallikes.index(he):
                return False
    return True
 
def matchmaker(guyprefers, galprefers, guys):
    guysfree = guys[:]
    engaged  = {}
    guyprefers2 = copy.deepcopy(guyprefers)
    galprefers2 = copy.deepcopy(galprefers)
    while guysfree:
        guy = guysfree.pop(0)
        guyslist = guyprefers2[guy]
        gal = guyslist.pop(0)
        fiance = engaged.get(gal)
        if not fiance:
            engaged[gal] = guy
        else:
            galslist = galprefers2[gal]
            if galslist.index(fiance) > galslist.index(guy):
                engaged[gal] = guy
                if guyprefers2[fiance]:
                    guysfree.append(fiance)
            else:
                if guyslist:
                    guysfree.append(guy)
    return engaged

# Matching CellphoneS với TGDD

In [6]:
cps_tgdd_name = name_matcher(cps_column, tgdd_column)
max_value = np.array(cps_tgdd_name).max()
cps_tgdd_name = np.array(cps_tgdd_name)/max_value

cps_tgdd_instance = instance_matcher(cps_df, tgdd_df, cps_column, tgdd_column)
cps_tgdd_instance = np.array(cps_tgdd_instance)

cps_tgdd = np.add(cps_tgdd_name, cps_tgdd_instance)/2
tgdd_cps = cps_tgdd.T

cps_tgdd = pd.DataFrame(cps_tgdd, columns=tgdd_column, index=cps_column)
tgdd_cps = pd.DataFrame(tgdd_cps, columns=cps_column, index=tgdd_column)
cps_tgdd

,ten,gia,cong_nghe_man_hinh,cpu,ram,bo_nho_trong,sim,pin,kich_thuoc_man_hinh
ten,0.485130,0.583333,0.944444,0.583333,0.583333,0.777778,0.583333,0.555556,0.972222
gia,0.583333,0.437853,0.944444,0.583333,0.583333,0.833333,0.555556,0.555556,0.972222
kich_thuoc_man_hinh,0.972222,0.972222,0.750000,0.972222,1.000000,0.888889,0.972222,0.972222,0.337209
cong_nghe_man_hinh,0.944444,0.944444,0.181818,0.956093,0.972222,0.833333,0.972222,0.944444,0.750000
chipset,0.666667,0.666667,0.944444,0.612222,0.694444,0.805556,0.666667,0.666667,0.972222
dung_luong_ram,0.861111,0.833333,0.888889,0.861111,0.416667,0.767094,0.861111,0.861111,0.916667
bo_nho_trong,0.777778,0.833333,0.833333,0.833333,0.738889,0.250000,0.833333,0.805556,0.888889
pin,0.555556,0.555556,0.944444,0.579812,0.583333,0.805556,0.555556,0.488372,0.972222
the_sim,0.638889,0.666667,0.888889,0.675926,0.666667,0.777778,0.611111,0.666667,0.916667


In [7]:
guyprefers = {}
for c in tgdd_column:
    guyprefers[c] = cps_tgdd.sort_values(by=[c]).index.to_list()

galprefers = {}
for c in cps_column:
    galprefers[c] = tgdd_cps.sort_values(by=[c]).index.to_list()

guys = sorted(guyprefers.keys())
engaged = matchmaker(guyprefers, galprefers, guys)
engaged

{'bo_nho_trong': 'bo_nho_trong',
 'cong_nghe_man_hinh': 'cong_nghe_man_hinh',
 'pin': 'pin',
 'gia': 'gia',
 'kich_thuoc_man_hinh': 'kich_thuoc_man_hinh',
 'dung_luong_ram': 'ram',
 'ten': 'ten',
 'chipset': 'cpu',
 'the_sim': 'sim'}

# Matching CellphoneS với Tiki

In [8]:
cps_tiki_name = name_matcher(cps_column, tiki_column)
max_value = np.array(cps_tiki_name).max()
cps_tiki_name = np.array(cps_tiki_name)/max_value

cps_tiki_instance = instance_matcher(cps_df, tiki_df, cps_column, tiki_column)
cps_tiki_instance = np.array(cps_tiki_instance)

cps_tiki = np.add(cps_tiki_name, cps_tiki_instance)/2
tiki_cps = cps_tiki.T

cps_tiki = pd.DataFrame(cps_tiki, columns=tiki_column, index=cps_column)
tiki_cps = pd.DataFrame(tiki_cps, columns=cps_column, index=tiki_column)
cps_tiki

,ten,gia,chipset,cong_nghe_man_hinh,pin,loai_sim,ram,rom,kich_thuoc_man_hinh
ten,0.495475,0.583333,0.666667,0.944444,0.555556,0.722222,0.583333,0.583333,0.972222
gia,0.583333,0.433225,0.666667,0.944444,0.555556,0.694444,0.583333,0.583333,0.972222
kich_thuoc_man_hinh,0.972222,0.972222,0.972222,0.750000,0.972222,0.916667,1.000000,0.972222,0.272727
cong_nghe_man_hinh,0.944444,0.944444,0.939068,0.379310,0.929293,0.874183,0.972222,0.944444,0.750000
chipset,0.666667,0.666667,0.441624,0.941339,0.663522,0.663542,0.694444,0.694444,0.972222
dung_luong_ram,0.861111,0.833333,0.888889,0.888889,0.861111,0.777778,0.513889,0.619048,0.884409
bo_nho_trong,0.777778,0.833333,0.805556,0.833333,0.805556,0.777778,0.638889,0.527778,0.888889
pin,0.555556,0.555556,0.664216,0.941020,0.478417,0.690996,0.583333,0.583333,0.972222
the_sim,0.638889,0.666667,0.633271,0.872760,0.649425,0.576628,0.666667,0.666667,0.916667


In [9]:
guyprefers = {}
for c in tiki_column:
    guyprefers[c] = cps_tiki.sort_values(by=[c]).index.to_list()

galprefers = {}
for c in cps_column:
    galprefers[c] = tiki_cps.sort_values(by=[c]).index.to_list()

guys = sorted(guyprefers.keys())
engaged = matchmaker(guyprefers, galprefers, guys)
engaged

{'chipset': 'chipset',
 'cong_nghe_man_hinh': 'cong_nghe_man_hinh',
 'gia': 'gia',
 'kich_thuoc_man_hinh': 'kich_thuoc_man_hinh',
 'the_sim': 'loai_sim',
 'pin': 'pin',
 'dung_luong_ram': 'ram',
 'bo_nho_trong': 'rom',
 'ten': 'ten'}

# Matching CellphoneS với Viettel

In [10]:
cps_vt_name = name_matcher(cps_column, vt_column)
max_value = np.array(cps_vt_name).max()
cps_vt_name = np.array(cps_vt_name)/max_value

cps_vt_instance = instance_matcher(cps_df, vt_df, cps_column, vt_column)
cps_vt_instance = np.array(cps_vt_instance)

cps_vt = np.add(cps_vt_name, cps_vt_instance)/2
vt_cps = cps_vt.T

cps_vt = pd.DataFrame(cps_vt, columns=vt_column, index=cps_column)
vt_cps = pd.DataFrame(vt_cps, columns=cps_column, index=vt_column)
cps_vt

,ten,gia,chip,ram,bo_nho_trong,pin,sim,kich_thuoc_man_hinh,cong_nghe_man_hinh
ten,0.442368,0.583333,0.611111,0.583333,0.777778,0.555556,0.583333,0.972222,0.944444
gia,0.583333,0.406699,0.583333,0.583333,0.833333,0.555556,0.555556,0.972222,0.944444
kich_thuoc_man_hinh,0.972222,0.972222,0.944444,1.000000,0.888889,0.972222,0.972222,0.255319,0.750000
cong_nghe_man_hinh,0.944444,0.944444,0.910643,0.972222,0.833333,0.944444,0.964026,0.750000,0.340909
chipset,0.666667,0.666667,0.563531,0.694444,0.805556,0.666667,0.663993,0.972222,0.941198
dung_luong_ram,0.861111,0.833333,0.888889,0.416667,0.781746,0.861111,0.861111,0.875000,0.888889
bo_nho_trong,0.777778,0.833333,0.805556,0.738889,0.233333,0.805556,0.833333,0.876068,0.833333
pin,0.555556,0.555556,0.580756,0.583333,0.805556,0.490196,0.552649,0.972222,0.940847
the_sim,0.638889,0.666667,0.632560,0.666667,0.777778,0.666667,0.602339,0.916667,0.868056


In [11]:
guyprefers = {}
for c in vt_column:
    guyprefers[c] = cps_vt.sort_values(by=[c]).index.to_list()

galprefers = {}
for c in cps_column:
    galprefers[c] = vt_cps.sort_values(by=[c]).index.to_list()

guys = sorted(guyprefers.keys())
engaged = matchmaker(guyprefers, galprefers, guys)
engaged

{'bo_nho_trong': 'bo_nho_trong',
 'chipset': 'chip',
 'cong_nghe_man_hinh': 'cong_nghe_man_hinh',
 'gia': 'gia',
 'kich_thuoc_man_hinh': 'kich_thuoc_man_hinh',
 'pin': 'pin',
 'dung_luong_ram': 'ram',
 'ten': 'ten',
 'the_sim': 'sim'}

# Matching CellphoneS với Didongviet

In [12]:
cps_ddv_name = name_matcher(cps_column, ddv_column)
max_value = np.array(cps_ddv_name).max()
cps_ddv_name = np.array(cps_ddv_name)/max_value

cps_ddv_instance = instance_matcher(cps_df, ddv_df, cps_column, ddv_column)
cps_ddv_instance = np.array(cps_ddv_instance)

cps_ddv = np.add(cps_ddv_name, cps_ddv_instance)/2
ddv_cps = cps_ddv.T

cps_ddv = pd.DataFrame(cps_ddv, columns=ddv_column, index=cps_column)
ddv_cps = pd.DataFrame(ddv_cps, columns=cps_column, index=ddv_column)
cps_ddv

,ten,gia,chip,man_hinh,cong_nghe_man_hinh,pin,sim,ram,rom
ten,0.471751,0.583333,0.611111,0.694444,0.944444,0.555556,0.583333,0.583333,0.583333
gia,0.583333,0.419831,0.583333,0.694444,0.944444,0.555556,0.555556,0.583333,0.583333
kich_thuoc_man_hinh,0.972222,0.972222,0.944444,0.603428,0.750000,0.972222,0.972222,1.000000,0.972222
cong_nghe_man_hinh,0.944444,0.944444,0.907407,0.777778,0.340909,0.930159,0.952991,0.972222,0.944444
chipset,0.666667,0.666667,0.550980,0.722222,0.941198,0.663561,0.663377,0.694444,0.694444
dung_luong_ram,0.861111,0.833333,0.888889,0.758681,0.888889,0.861111,0.861111,0.455556,0.739583
bo_nho_trong,0.777778,0.833333,0.805556,0.735714,0.833333,0.805556,0.833333,0.743056,0.368687
pin,0.555556,0.555556,0.580303,0.666667,0.940847,0.463504,0.551906,0.583333,0.583333
the_sim,0.638889,0.666667,0.628889,0.666667,0.868056,0.650538,0.505848,0.666667,0.666667


In [13]:
guyprefers = {}
for c in ddv_column:
    guyprefers[c] = cps_ddv.sort_values(by=[c]).index.to_list()

galprefers = {}
for c in cps_column:
    galprefers[c] = ddv_cps.sort_values(by=[c]).index.to_list()

guys = sorted(guyprefers.keys())
engaged = matchmaker(guyprefers, galprefers, guys)
engaged

{'chipset': 'chip',
 'cong_nghe_man_hinh': 'cong_nghe_man_hinh',
 'gia': 'gia',
 'kich_thuoc_man_hinh': 'man_hinh',
 'pin': 'pin',
 'dung_luong_ram': 'ram',
 'bo_nho_trong': 'rom',
 'the_sim': 'sim',
 'ten': 'ten'}

# Matching CellphoneS với Điện máy xanh

In [14]:
cps_dmx_name = name_matcher(cps_column, dmx_column)
max_value = np.array(cps_dmx_name).max()
cps_dmx_name = np.array(cps_dmx_name)/max_value

cps_dmx_instance = instance_matcher(cps_df, dmx_df, cps_column, dmx_column)
cps_dmx_instance = np.array(cps_dmx_instance)

cps_dmx = np.add(cps_dmx_name, cps_dmx_instance)/2
dmx_cps = cps_dmx.T

cps_dmx = pd.DataFrame(cps_dmx, columns=dmx_column, index=cps_column)
dmx_cps = pd.DataFrame(dmx_cps, columns=cps_column, index=dmx_column)
cps_dmx

,ten,gia,chip,kich_thuoc_man_hinh,cong_nghe_man_hinh,pin,sim,ram,rom
ten,0.490741,0.583333,0.611111,0.972222,0.944444,0.555556,0.583333,0.583333,0.583333
gia,0.583333,0.452778,0.583333,0.972222,0.944444,0.555556,0.555556,0.583333,0.583333
kich_thuoc_man_hinh,0.972222,0.972222,0.944444,0.337209,0.750000,0.972222,0.972222,1.000000,0.972222
cong_nghe_man_hinh,0.944444,0.944444,0.916667,0.750000,0.181818,0.944444,0.972222,0.972222,0.944444
chipset,0.666667,0.666667,0.553333,0.972222,0.944444,0.666667,0.666667,0.694444,0.694444
dung_luong_ram,0.861111,0.833333,0.888889,0.916667,0.888889,0.861111,0.861111,0.472222,0.833333
bo_nho_trong,0.777778,0.833333,0.805556,0.888889,0.833333,0.805556,0.833333,0.772222,0.577778
pin,0.555556,0.555556,0.583333,0.972222,0.944444,0.484375,0.555556,0.583333,0.583333
the_sim,0.638889,0.666667,0.638889,0.916667,0.888889,0.666667,0.611111,0.666667,0.666667


In [15]:
guyprefers = {}
for c in dmx_column:
    guyprefers[c] = cps_dmx.sort_values(by=[c]).index.to_list()

galprefers = {}
for c in cps_column:
    galprefers[c] = dmx_cps.sort_values(by=[c]).index.to_list()

guys = sorted(guyprefers.keys())
engaged = matchmaker(guyprefers, galprefers, guys)
engaged

{'chipset': 'chip',
 'cong_nghe_man_hinh': 'cong_nghe_man_hinh',
 'gia': 'gia',
 'kich_thuoc_man_hinh': 'kich_thuoc_man_hinh',
 'pin': 'pin',
 'dung_luong_ram': 'ram',
 'bo_nho_trong': 'rom',
 'ten': 'ten',
 'the_sim': 'sim'}

In [16]:
cps_df = cps_df.rename(columns={"chipset": "cpu", "dung_luong_ram": "ram", "the_sim": "sim"})
tiki_df = tiki_df.rename(columns={"chipset": "cpu", "loai_sim": "sim", "rom": "bo_nho_trong"})
vt_df = vt_df.rename(columns={"chip": "cpu", "rom": "bo_nho_trong"})
ddv_df = ddv_df.rename(columns={"man_hinh": "kich_thuoc_man_hinh", "chip": "cpu", "rom": "bo_nho_trong"})
dmx_df = dmx_df.rename(columns={"chip": "cpu", "rom": "bo_nho_trong"})

In [17]:
cps_df['id'] = cps_df.index
tgdd_df['id'] = tgdd_df.index
tiki_df['id'] = tiki_df.index
vt_df['id'] = vt_df.index
ddv_df['id'] = ddv_df.index
dmx_df['id'] = dmx_df.index

In [18]:
tgdd_df.to_csv('clean_data/tgdd.csv', index=False)
cps_df.to_csv('clean_data/cellphones.csv', index=False)
tiki_df.to_csv('clean_data/tiki.csv', index=False)
vt_df.to_csv('clean_data/viettel.csv', index=False)
ddv_df.to_csv('clean_data/didongviet.csv', index=False)
dmx_df.to_csv('clean_data/dienmayxanh.csv', index=False)